In [16]:
from unittest.mock import inplace

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
import matplotlib.pyplot as plt
import re
%matplotlib inline
pd.set_option('future.no_silent_downcasting', True)
pd.options.mode.copy_on_write = True

data = pd.DataFrame( pd.read_excel('./breast-cancer.xls') )  # Data Import from local/ Drive file

data['inv-nodes'].to_string(index=False)

################# Data sanitisation #################
total_rows = len(data)
print("Data size before sanitisation, Total: ", total_rows)
data.dropna() # Drops all NA cells by row and saves it back to source data


print("Removed, Rows with NA: ", total_rows - len(data))
for x in data.index:

    if re.search( r"^[0-9]{1,2}-[0-9]{1,2}$",  str(data.loc[x, "age"]) ) is None and not ( re.search( r"^[0-9]{1,2}$",  str(data.loc[x, "age"])is None) ): #Using RegEx to confirm age pattern is not "XX-XX" / "X-XX" and not "XX"
        data.drop(index=x, inplace=True)
    elif re.search(r"^(ge40)$|^(premeno)$|^(lt40)$", str(data.loc[x, "menopause"])) is None:  #Using RegEx to confirm menopause name is only 1 of three
        data.drop(index=x, inplace=True)
    elif re.search(r"^[0-9]{1,2}-[0-9]{1,2}$", str(data.loc[x, "tumor-size"]) ) is None:  #Using RegEx to confirm tumor size is formated correctly
        data.drop(index=x, inplace=True)
    elif re.search(r"^[0-9]{1,2}-[0-9]{1,2}$", str(data.loc[x, "inv-nodes"])) is None: #Using RegEx to confirm node-caps is only yes or no Upper or lower case
        data.drop(index=x, inplace=True)
    elif re.match(r'^([Yy][eE][Ss]|[Nn][oO])$', str(data.loc[x, "node-caps"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x, inplace=True)
    elif re.search(r"^1$|^2$|^3$", str(data.loc[x, "deg-malig"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x, inplace=True)
    elif re.search(r"^([Ll][eE][fF][tT]|[Rr][Ii][Gg][hH][tT])$", str(data.loc[x, "breast"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x)
    elif re.search(r"^(left_up|left_low|central|right_up|right_low)$", str(data.loc[x, "breast-quad"]), re.IGNORECASE) is None:
        data.drop(index=x)#Using RegEx to confirm inv-nodes count is a number range

    #Force only standard answer for this query, Yes / Recurence will = Recurrence-event, anything with no / non = Non-recurrence-event
    elif re.search(r"(non|no|recurrence|Yes)", str(data.loc[x, "Class"]), re.IGNORECASE) is None:
        data.loc[x, "Class"] = "Non-recurrence-event"
        if re.search(r"(non|no)", str(data.loc[x, "Class"]), re.IGNORECASE) :
            data.loc[x, "Class"] = "Non-recurrence-event"
        else:
            data.loc[x, "Class"] = "Recurrence-event"
        data.drop(index=x)

data.reset_index(drop=True, inplace=True)  # Re indexes entire dataframe to fix deleted rows
print("Total removal rows: " , total_rows - len(data))
################# Data sanitisation #################
print(data)
################# Data Formating #################
def range_median(column):
    main_array = []
    for y in data.index:
        value = str(data.loc[y, column])
        if re.search(r"^[0-9]{1,2}-[0-9]{1,2}$", str(value)) is not None:
            array = np.array(re.findall(r"[0-9]{1,2}", value), dtype=int)
            main_array.append(np.median(array))

        elif  re.search( r"^[0-9]{1,2}$",  value)is not None:
            array = np.array(re.findall(r"[0-9]{1,2}", value), dtype=int)
            main_array.append(np.median(array))
    data[str(column)] = main_array
    return main_array


def uniques_translator(column, inpalce_trigger):
    column_uniques = np.array(pd.unique(data[column]))
    translation_dictionary = {}
    for x in range(column_uniques.size):
        translation_dictionary.update({x : column_uniques[x]})
    if inpalce_trigger == True:
        data[str(column)] = data[[str(column)]].replace(column_uniques, translation_dictionary.keys())
        return data[[str(column)]]
    else:
        return data[[str(column)]].replace(column_uniques, translation_dictionary) , translation_dictionary

##  Dictionary of the change to revert back later    ##


numeric_Translation_Menopause = uniques_translator('menopause', False)
numeric_Translation_NodeCaps = uniques_translator('node-caps', False)
numeric_Translation_Breast = uniques_translator('breast', False)
numeric_Translation_irradiat = uniques_translator('irradiat', False)
numeric_Translation_Class = uniques_translator('Class', False)

##  Dictionary of the change to revert back later    ##

## Translating all string columns to int equivilant  ##

uniques_translator('menopause', True)
uniques_translator('node-caps', True)
uniques_translator('breast-quad', True)
uniques_translator('breast', True)
uniques_translator('irradiat', True)
uniques_translator('Class', True)
## Translating all string columns to int equivilant  ##
#range_median('age')


## Range medium finder for all non int only columns
range_median('age')
range_median('tumor-size')
range_median('inv-nodes')


################# Data Formating #################

################# Training Data Masking #################
training_data_per =  90/100.0

data['train'] = np.random.rand(len(data)) < training_data_per # Creates random array for each row  of Data, true or false using the fraction
Training_Data = data[data.train == 1] # Appends train Data with the array for each row, effectively showing which row at random will be removed

Training_Data = Training_Data.drop('train', axis=1).sample(frac=1)
################# Data Test Masking #################

################# Testing Data Seperation / Masking #################
Testing_Data = data[data.train == 0]
Testing_Data.drop('train', axis=1, inplace=True)

Training_Data = data[data.train == 1]
Training_Data.drop('train', axis=1, inplace=True)

X = Training_Data.to_numpy()    ### OR .values[:,:4]
targets = [[1,0,0],[0,1,0],[0,0,1]]
Y = np.array([targets[int(x)] for x in train.values[:,4:5]])
print(X)
################# Testing Data Masking #################
################# Testing Data Masking #################

#data.sample(len(data))

Data size before sanitisation, Total:  286
Removed, Rows with NA:  0
Total removal rows:  99
       age menopause tumor-size inv-nodes node-caps  deg-malig breast  \
0    40-49   premeno      15-19       0-2       yes          3  right   
1    50-59      ge40      15-19       0-2        no          1  right   
2    50-59      ge40      35-39       0-2        no          2   left   
3    40-49   premeno      35-39       0-2       yes          3  right   
4    50-59      ge40      40-44       0-2        no          3   left   
..     ...       ...        ...       ...       ...        ...    ...   
182  40-49   premeno      35-39       0-2        no          2  right   
183  40-49   premeno      20-24       0-2        no          2  right   
184  60-69      ge40      15-19       0-2        no          3  right   
185  50-59   premeno      15-19       0-2        no          2  right   
186  50-59      ge40      40-44       0-2        no          3   left   

    breast-quad irradiat      

C:\Users\vladl\AppData\Local\Temp\ipykernel_9520\681451056.py:131: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Y = np.array([targets[int(x)] for x in train.values[:,4:5]])
